<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B6%D0%B5%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание жестов
# Gesture recognition

В этом уроке научимся распознавать жесты в ограничивающей рамке.




In this lesson, we will learn to recognize gestures in a bounding box.

In [1]:
# импортируем необходимые модули
# import the necessary packages
import numpy as np
import imutils
import cv2
from sklearn.metrics import pairwise

### Загрузка файлов
### Uploading files

In [2]:
!wget http://dataudt.ru/datasets/cv/Lesson_44.Recognizing_gestures.zip
!unzip Lesson_44.Recognizing_gestures.zip
%cd /content/Lesson_44.Recognizing_gestures

--2022-02-08 09:04:00--  http://dataudt.ru/datasets/cv/Lesson_44.Recognizing_gestures.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3566698 (3.4M) [application/zip]
Saving to: ‘Lesson_44.Recognizing_gestures.zip’

Lesson_44.Recognizi 100%[===================>]   3.40M  1.40MB/s    in 2.4s    

2022-02-08 09:04:04 (1.40 MB/s) - ‘Lesson_44.Recognizing_gestures.zip’ saved [3566698/3566698]

Archive:  Lesson_44.Recognizing_gestures.zip
  inflating: Lesson_44.Recognizing_gestures/detect.py  
   creating: Lesson_44.Recognizing_gestures/detect/
 extracting: Lesson_44.Recognizing_gestures/detect/__init__.py  
   creating: Lesson_44.Recognizing_gestures/detect/__pycache__/
  inflating: Lesson_44.Recognizing_gestures/detect/__pycache__/__init__.cpython-38.pyc  
   creating: Lesson_44.Recognizing_gestures/detect/gesture_recognition/
  inflating: Lesson_44.Recogni

In [3]:
# явно укажем аргументы необходимые для работы
# video - путь к видеофайлу
# bounding_box - ограничивающая рамка
# output - выходное видео  

# explicitly specify the arguments needed to work
# video - path to the video file
# bounding_box - bounding box
# output - output video 
args = {
	"video": "recognize_in.mp4",
    "bounding_box": "140,50,415,345",
	"output" : "output.avi"
}

In [4]:
class GestureDetector:
    def __init__(self):
        pass

    def detect(self, thresh, cnt):
        # вычисляем выпуклую оболочку контура, а затем 
        # находим самые крайние точки вдоль корпуса
        # compute the convex hull of the contour and then find the extreme most points along
        # the hull
        hull = cv2.convexHull(cnt)
        extLeft = tuple(hull[hull[:, :, 0].argmin()][0])
        extRight = tuple(hull[hull[:, :, 0].argmax()][0])
        extTop = tuple(hull[hull[:, :, 1].argmin()][0])
        extBot = tuple(hull[hull[:, :, 1].argmax()][0])

        # вычисляем центральные x, y координаты на основе крайних точек,
        # затем добавляем небольшой дополнительный процент к координате y,
        # чтобы опустить область к центру ладони
        # compute the center (x, y)-coordinates based on the extreme points, then add a little
        # extra percentage to the y-coordinate to lower the region towards the center of the
        # palm
        cX = (extLeft[0] + extRight[0]) // 2
        cY = (extTop[1] + extBot[1]) // 2
        cY += (cY * 0.15)
        cY = int(cY)

        # вычислим расстояния между центроидом и крайними координатами, 
        # затем найдём наибольшее расстояние и используем его 
        # для получения полного радиуса области ладони
        # compute the distances between the centroid and the extreme coordinates, then find the
        # largest distance, and use it to complete radius of palm region
        D = pairwise.euclidean_distances([(cX, cY)], Y=[extLeft, extRight, extTop, extBot])[0]
        maxDist = D[D.argmax()]
        r = int(0.7 * maxDist)
        circum = 2 * np.pi * r # длинна окружности

        # построим круговую ИОИ, включающую ладонь + пальцы
        # construct the circular ROI that includes the palm + fingers
        circleROI = np.zeros(thresh.shape[:2], dtype="uint8")
        cv2.circle(circleROI, (cX, cY), r, 255, 1)
        circleROI = cv2.bitwise_and(thresh, thresh, mask=circleROI)

        # находим контуры в круговой области интереса 
        # и инициализируем общее количество пальцев, 
        # подсчитанных в кадре
        # find contours in the circular ROI and initialize the total number of fingers counted
        # in the frame
        cnts = cv2.findContours(circleROI.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnts = imutils.grab_contours(cnts)
        total = 0

        # перебираем контуры
        # loop over the contours
        for c in cnts:
            # вычисляем ограничивающую рамку контура
            # compute the bounding box of the contour
            (x, y, w, h) = cv2.boundingRect(c)

            # увеличиваем общее количество пальцев только в том случае, 
            # если (1) количество точек по контуру не превышает 25% окружности 
            # и (2) контурная область не находится в нижней части круга
            # (которая является областью запястья)
            # increment the total number of fingers only if (1) the number of points along the
            # contour does not exceed 25% of the circumfrence and (2) the contour region is not
            # at the bottom of the circle (which is the wrist area)
            if c.shape[0] < circum * 0.25 and (y + h) < cY + (cY * 0.25):
                total += 1

        # возвращаем общее количество обнаруженных пальцев
        # return the total number of fingers detected
        return total

    @staticmethod
    def drawText(roi, i, val, color=(0, 0, 255)):
        # рисуем текст на ИОИ
        # draw the text on the ROI
        cv2.putText(roi, str(val), ((i * 50) + 20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
            color, 3)

    @staticmethod
    def drawBox(roi, i, color=(0, 0, 255)):
        # рисуем рамку на ИОИ
        # draw the box on the ROI
        cv2.rectangle(roi, ((i * 50) + 10, 10), ((i * 50) + 50, 60), color, 2)

In [5]:
class MotionDetector:
	def __init__(self, accumWeight=0.5):
        # сохраним накопленный весовой коэффициент
		# store the accumulated weight factor
		self.accumWeight = accumWeight

        # инициализируем фоновую модель
		# initialize the background model
		self.bg = None

	def update(self, image):
        # если фоновой модель нет, инициализируем её
		# if the background model is None, initialize it
		if self.bg is None:
			self.bg = image.copy().astype("float")
			return

        # обновим фоновую модель путем накопления средневзвешенного значения
		# update the background model by accumulating the weighted average
		cv2.accumulateWeighted(image, self.bg, self.accumWeight)

	def detect(self, image, tVal=25):
        # вычисляем абсолютную разницу между фоновой моделью 
        # и переданным изображением, затем 
        # пороговое значение дельта-изображения
		# compute the absolute difference between the background model and the image
		# passed in, then threshold the delta image
		delta = cv2.absdiff(self.bg.astype("uint8"), image) # разница
		thresh = cv2.threshold(delta, tVal, 255, cv2.THRESH_BINARY)[1] # порог

        # найдём контуры в пороговом изображении
		# find contours in the thresholded image
		cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
		cnts = imutils.grab_contours(cnts)

        # если контуры не найдены, возвращаем None
		# if no contours were found, return None
		if len(cnts) == 0:
			return None

        # кортеж изображения с пороговым значением
        # вместе с контурной областью
		# otherwise, return a tuple of the thresholded image along with the contour area
        # в противном случае возвращаем 
		return (thresh, max(cnts, key=cv2.contourArea))

In [6]:
# загружаем видео и указываем путь к выходному файлу
# load the video and initialize pointer to output video file
camera = cv2.VideoCapture(args["video"])
writer = None
# распакуем интересующую область изображения для руки,
# затем инициализируем детектор движения 
# и детектор жестов
# unpack the hand ROI, then initialize the motion detector and gesture detector
(top, right, bot, left) = np.int32(args["bounding_box"].split(","))
gd = GestureDetector()
md = MotionDetector()

# инициализируем общее количество фреймов, 
# прочитанных на данный момент, переменную учёта, 
# которая используется для отслеживания количества 
# последовательных фреймов, в которых появился жест, 
# вместе со значениями, распознаваемыми детектором жестов
# initialize the total number of frames read thus far, a bookkeeping variable used to
# keep track of the number of consecutive frames a gesture has appeared in, along
# with the values recognized by the gesture detector
numFrames = 0
gesture = None
values = []

# бесконечный цикл                                 
# keep looping
while True:
    # захватим текущий кадр                                              
    # grab the current frame
    (grabbed, frame) = camera.read()

    # если мы просматриваем видео и не захватили кадр, 
    # значит, мы достигли конца видео
    # if we are viewing a video and we did not grab a frame, then we have reached the
    # end of the video
    if args.get("video") and not grabbed:
        break

    # изменим размер кадра
    # resize the frame and flip it so the frame is no longer a mirror view
    frame = imutils.resize(frame, width=640)
    # если используем изображение с вебкамеры, то отзеркалим кадр для удобства
    # if we use a webcam image, we'll mirror the frame for convenience
    if not args.get("video", False):
        frame = cv2.flip(frame, 1)
    clone = frame.copy()
    (frameH, frameW) = frame.shape[:2]

    # извлекаем ИОИ, переходя справа налево, 
    # так как изображение зеркально, 
    # затем слегка размываем его                                       
    # extract the ROI, passing in right:left since the image is mirrored, then
    # blur it slightly
    roi = frame[top:bot, right:left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # если мы не набрали 32 начальных кадра, 
    # то откалибруем детектор кожи                                     
    # if we not reached 32 initial frames, then calibrate the skin detector
    if numFrames < 32:
        md.update(gray)

    # в противном случае обнаруживаем кожу в ИОИ                                          
    # otherwise, detect skin in the ROI
    else:
        # обнаруживаем движение (то есть кожу) на изображении   
        # detect motion (i.e., skin) in the image
        skin = md.detect(gray)

        # проверяем, была ли обнаружена кожа                                      
        # check to see if skin has been detected
        if skin is not None:
            # распаковываем кортеж и обнаруживаем жест в пороговом изображении
            # unpack the tuple and detect the gesture in the thresholded image
            (thresh, c) = skin
            cv2.drawContours(clone, [c + (right, top)], -1, (0, 255, 0), 2)
            fingers = gd.detect(thresh, c)

            # если текущий счетчик жестов не создан, инициализируем его
            # if the current gesture count is None, initialize it
            if gesture is None:
                gesture = [1, fingers]

            # иначе счетчик пальцев был инициализирован
            # otherwise the finger count has been initialized
            else:
                # если количество пальцев одинаковое, 
                # увеличиваем количество кадров
                # if the finger counts are the same, increment the number of frames
                if gesture[1] == fingers:
                    gesture[0] += 1

                    # если набрали достаточное количество кадров, 
                    # рисуем номер на экране
                    # if we have reached a sufficient number of frames, draw the number of the
                    # screen
                    if gesture[0] >= 25:
                        # если список значений уже заполнен, сбросим его
                        # if the values list is already full, reset it
                        if len(values) == 2:
                            values = []

                        # обновим список значений и сбросим детектор жестов
                        # update the values list and reset the gesture
                        values.append(fingers)
                        gesture = None

                # в противном случае счетчики пальцев не совпадают, 
                # поэтому сбросим учетную переменную
                # otherwise, the finger counts do not match up, so reset the bookkeeping variable
                else:
                    gesture = None

    # проверяем, есть ли хотя бы одна запись в списке значений
    # check to see if there is at least one entry in the values list
    if len(values) > 0:
        # рисуем первую цифру и знак плюса
        # draw the first digit and the plus sign
        GestureDetector.drawBox(clone, 0)
        GestureDetector.drawText(clone, 0, values[0])
        GestureDetector.drawText(clone, 1, "+")

    # проверяем, есть ли вторая запись в списке значений
    # check to see if there is a second entry in the values list
    if len(values) == 2:
        # рисуем вторую цифру, знак равенства и, наконец, ответ
        # draw the second digit, the equal sign, and fianlly the answer
        GestureDetector.drawBox(clone, 2)
        GestureDetector.drawText(clone, 2, values[1])
        GestureDetector.drawText(clone, 3, "=")
        GestureDetector.drawBox(clone, 4, color=(0, 255, 0))
        GestureDetector.drawText(clone, 4, values[0] + values[1], color=(0, 255, 0))

    # рисуем интересующую область изображения для руки
    # и увеличиваем количество обработанных кадров 
    # draw the hand ROI and increment the number of processed frames
    cv2.rectangle(clone, (left, top), (right, bot), (0, 0, 255), 2)
    numFrames += 1

    # показываем исходный и скопированный кадр
    # check if the video writer is None
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(clone.shape[1], clone.shape[0]), True)
  
    # если видео есть, записываем кадр с распознанным движением на диск
	# if the writer is not None, write the frame with recognized
	# faces to disk
    if writer is not None:
        writer.write(clone)
  
    # увеличиваем счетчик кадров
	# increment the frame counter
    numFrames += 1

# очищаем данные камеры
# cleanup the camera
camera.release()

# проверяем, нужно ли очистить данные
# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Выполнение ячейки может занять некоторое время. Наше выходное видео создается в формате .avi. Нам нужно преобразовать его в формат .mp4.

Cell execution may take some time. Our output video is created in .avi format. We need to convert it to .mp4 format.

In [7]:
!ffmpeg -i "output.avi" output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [8]:
#@title Воспроизводим выходное видео
#@title Display video inline
from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)